In [ ]:
import pandas as pd
import requests
import time
import math

# === API KEY ===
API_KEY = '*****'  # ⬅️ Reemplaza por tu clave válida

# === DESTINO FIJO UDA ===
DESTINO = "-2.918554,-79.001596"

# === LEER Y LIMPIAR DATOS ===
df = pd.read_csv("reporte_codificado.csv", sep=";", encoding="latin1")
df.columns = [col.strip().upper() for col in df.columns]
df['LATITUD'] = df['LATITUD'].astype(str).str.replace(',', '.').astype(float)
df['LONGITUD'] = df['LONGITUD'].astype(str).str.replace(',', '.').astype(float)
df['ORIGEN_COORD'] = df['LATITUD'].astype(str) + ',' + df['LONGITUD'].astype(str)

# === AGRUPAR EN LOTES DE 25 ===
batch_size = 25
num_batches = math.ceil(len(df) / batch_size)

distancias_km = []
valid_index = []
descartados_index = []

print("📡 Consultando por lotes...")

for i in range(num_batches):
    batch_df = df.iloc[i*batch_size:(i+1)*batch_size].copy()
    origenes = '|'.join(batch_df['ORIGEN_COORD'])

    url = (
        f"https://maps.googleapis.com/maps/api/distancematrix/json"
        f"?origins={origenes}"
        f"&destinations={DESTINO}"
        f"&mode=driving"
        f"&key={API_KEY}"
    )

    try:
        resp = requests.get(url)
        data = resp.json()

        for j, element in enumerate(data['rows']):
            idx = batch_df.index[j]
            try:
                if element['elements'][0]['status'] == 'OK':
                    distancia = element['elements'][0]['distance']['value'] / 1000
                    if 0.8 < distancia < 30:
                        distancias_km.append(distancia)
                        valid_index.append(idx)
                    else:
                        descartados_index.append(idx)
                else:
                    descartados_index.append(idx)
            except:
                descartados_index.append(idx)

    except Exception as e:
        print(f"⚠️ Error en batch {i}: {e}")
        continue

    time.sleep(1.5)

# === EXPORTAR RESULTADOS ===
df_validos = df.loc[valid_index, ['PERSONA', 'LATITUD', 'LONGITUD']].copy()
df_validos['DISTANCIA_GOOGLE_KM'] = distancias_km

df_descartados = df.loc[descartados_index, ['PERSONA', 'LATITUD', 'LONGITUD']].copy()

df_validos.to_excel("reporte_google_con_distancia.xlsx", index=False)
df_descartados.to_excel("reporte_google_descartados.xlsx", index=False)

print("✅ Procesamiento finalizado:")
print(f"   - Registros válidos: {len(df_validos)}")
print(f"   - Registros descartados: {len(df_descartados)}")
print("📁 Archivos exportados correctamente.")
